In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
print(os.getcwd())

/content


In [ ]:
os.chdir('drive/MyDrive')

In [ ]:
os.chdir('cse_571_grasp')

In [ ]:
pip install open3d

In [ ]:
pip install trimesh

In [ ]:
pip install rtree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 535.2/535.2 kB 10.0 MB/s eta 0:00:00


In [ ]:
import trimesh
import numpy as np

def load_gripper_mesh():
    base = trimesh.load("panda_gripper/hand.stl")
    finger_l = trimesh.load("panda_gripper/finger.stl")
    finger_r = finger_l.copy()

    finger_l.apply_transform(trimesh.transformations.euler_matrix(0, 0, np.pi))
    finger_l.apply_translation([0.04, 0, 0.0584])
    finger_r.apply_translation([-0.04, 0, 0.0584])

    fingers = trimesh.util.concatenate([finger_l, finger_r])
    hand = trimesh.util.concatenate([fingers, base])
    return hand


In [ ]:
import open3d as o3d

In [ ]:
os.getcwd()

'/content/drive/MyDrive/cse_571_grasp'

In [ ]:
os.chdir('MyDrive/cse_571_grasp')

In [ ]:
os.chdir('../../')

In [ ]:
import pickle

with open('./pc_data', 'rb') as f:
    pc_data, grasps_data, labels_data = pickle.load(f)

In [ ]:
with open('./test_pc_data','rb') as f:
  test_pc_data, test_grasp_data, test_labels_data = pickle.load(f)

In [ ]:
def normalize_point_cloud(points, fixed_size=1024):
    if len(points) < fixed_size:

        padding = np.zeros((fixed_size - len(points), 3))
        points = np.vstack((points, padding))
    elif len(points) > fixed_size:
        indices = np.random.choice(len(points), fixed_size, replace=False)
        points = points[indices]
    return points

def normalize_pc_data(pc_data, fixed_size=1024):
    normalized_pc_data = [normalize_point_cloud(points, fixed_size) for points in pc_data]
    return normalized_pc_data

fixed_size = 1024
normalized_pc_data = normalize_pc_data(pc_data, fixed_size)
normalized_test_data = normalize_pc_data(test_pc_data, fixed_size)

In [ ]:
print(len(pc_data[0]))
print(len(pc_data[1]))
print(len(normalized_pc_data[1]))
print(len(normalized_test_data[1]))

53
28
1024
1024


In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

class GraspDataset(Dataset):
    def __init__(self, point_clouds, grasps, labels, region_size=0.1, max_points=1024):
        self.point_clouds = point_clouds
        self.grasps = grasps
        self.labels = labels
        self.region_size = region_size
        self.max_points = max_points

    def __len__(self):
        return len(self.point_clouds)

    def transform_to_local(self, point_cloud, grasp):

        transformation_matrix = np.linalg.inv(grasp)
        homogenous_coords = np.hstack((point_cloud, np.ones((point_cloud.shape[0], 1))))
        transformed_coords = homogenous_coords @ transformation_matrix.T
        return transformed_coords[:, :3]

    def normalize(self, point_cloud):
        centroid = np.mean(point_cloud, axis=0)
        point_cloud -= centroid
        max_dist = np.max(np.sqrt(np.sum(point_cloud ** 2, axis=1)))
        point_cloud /= max_dist
        return point_cloud

    def __getitem__(self, idx):
      point_cloud = self.point_clouds[idx]
      grasp = self.grasps[idx]
      label = self.labels[idx]


      transformed_point_cloud = self.transform_to_local(point_cloud, grasp)


      normalized_point_cloud = self.normalize(transformed_point_cloud)

      normalized_point_cloud = normalized_point_cloud[:, :3]

      return torch.from_numpy(normalized_point_cloud), torch.tensor(label)




def collate_fn(batch):
    max_points = max(item[0].shape[0] for item in batch)
    max_points = min(max_points, 1024)

    point_clouds = []
    labels = []

    for point_cloud, label in batch:
        if point_cloud.shape[0] > max_points:
            point_cloud = point_cloud[:max_points, :]
        else:
            padding = max_points - point_cloud.shape[0]
            point_cloud = torch.cat([point_cloud, torch.zeros(padding, 3)], dim=0)


        point_cloud = point_cloud.permute(1, 0).unsqueeze(0)
        point_clouds.append(point_cloud)
        labels.append(label)

    point_clouds = torch.cat(point_clouds, dim=0)
    labels = torch.tensor(labels, dtype=torch.float32)

    return point_clouds, labels


In [ ]:
dataset = GraspDataset(normalized_pc_data, grasps_data, labels_data)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)


In [ ]:
test_dataset = GraspDataset(normalized_test_data, test_grasp_data, test_labels_data)
test_dataloader = DataLoader(test_dataset, batch_size = 32, shuffle = True, collate_fn = collate_fn)

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader

class PointNet(nn.Module):
    def __init__(self):
        super(PointNet, self).__init__()
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 1)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)

        self.mp1 = nn.AdaptiveMaxPool1d(1)

    def forward(self, x):
        x = x.transpose(2, 1)
        x = self.relu(self.bn1(self.conv1(x.float())))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.mp1(x)
        x = x.view(-1, 1024)
        x = self.dropout(self.relu(self.bn4(self.fc1(x))))
        x = self.dropout(self.relu(self.bn5(self.fc2(x))))
        x = self.fc3(x)
        return x

def train_and_evaluate(net, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100, patience=10):
    best_loss = float('inf')
    early_stop_counter = 0
    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        net.train()
        running_loss = 0.0
        num_batches = 0

        for i, (inputs, labels) in enumerate(train_loader, 0):
            inputs = inputs.transpose(2, 1).to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = net(inputs).squeeze()

            if torch.isnan(outputs).any():
                print("NaN values found in outputs")
                continue

            loss = criterion(outputs, labels)

            if torch.isnan(loss).any():
                print("NaN values found in loss")
                continue

            loss.backward()
            torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=1.0)
            optimizer.step()

            running_loss += loss.item()
            num_batches += 1


        train_losses.append(running_loss / num_batches)

        scheduler.step()

        # Validation phase
        net.eval()
        val_running_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.transpose(2, 1).to(device)
                labels = labels.to(device)
                outputs = net(inputs).squeeze()
                loss = criterion(outputs, labels)
                val_running_loss += loss.item()

                predicted = (outputs > 0.5).float()
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_losses.append(val_running_loss / len(val_loader))
        accuracy = 100 * correct / total

        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}, Val Accuracy: {accuracy:.2f}%")

        # Early stopping
        if val_running_loss < best_loss:
            best_loss = val_running_loss
            early_stop_counter = 0
        else:
            early_stop_counter += 1

        if early_stop_counter >= patience:
            print("Early stopping triggered")
            break

    print("Finished Training")
    return train_losses, val_losses

learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = PointNet().to(device)
optimizer = optim.Adam(net.parameters(), lr=learning_rate, weight_decay=1e-4)  # Add weight decay
criterion = nn.BCEWithLogitsLoss()
scheduler = StepLR(optimizer, step_size=20, gamma=0.5)  # Reduce LR every 20 epochs


train_losses, val_losses = train_and_evaluate(net, dataloader, test_dataloader, criterion, optimizer, scheduler, num_epochs=100)

# Plotting the training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss over Epochs')
plt.show()


<ipython-input-16-9c4dce6a0018>:45: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return torch.from_numpy(normalized_point_cloud), torch.tensor(label)


Epoch [1/100], Train Loss: 0.6060, Val Loss: 0.5747, Val Accuracy: 59.74%
Epoch [2/100], Train Loss: 0.5912, Val Loss: 0.5696, Val Accuracy: 65.51%
Epoch [3/100], Train Loss: 0.5880, Val Loss: 0.5655, Val Accuracy: 67.11%
Epoch [4/100], Train Loss: 0.5860, Val Loss: 0.5616, Val Accuracy: 65.46%
Epoch [5/100], Train Loss: 0.5843, Val Loss: 0.5692, Val Accuracy: 61.70%
Epoch [6/100], Train Loss: 0.5838, Val Loss: 0.5628, Val Accuracy: 64.30%
Epoch [7/100], Train Loss: 0.5830, Val Loss: 0.5692, Val Accuracy: 66.49%
Epoch [8/100], Train Loss: 0.5823, Val Loss: 0.5674, Val Accuracy: 65.50%
Epoch [9/100], Train Loss: 0.5807, Val Loss: 0.5653, Val Accuracy: 66.99%
Epoch [10/100], Train Loss: 0.5812, Val Loss: 0.5656, Val Accuracy: 69.02%
Epoch [11/100], Train Loss: 0.5802, Val Loss: 0.5691, Val Accuracy: 68.33%
Epoch [12/100], Train Loss: 0.5790, Val Loss: 0.5661, Val Accuracy: 67.83%


In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn

class PointNet(nn.Module):
    def __init__(self):
        super(PointNet, self).__init__()
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 1)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)

        self.mp1 = nn.AdaptiveMaxPool1d(1)

    def forward(self, x):
        x = x.transpose(2, 1)
        x = self.relu(self.bn1(self.conv1(x.float())))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.mp1(x)
        x = x.view(-1, 1024)
        x = self.dropout(self.relu(self.bn4(self.fc1(x))))
        x = self.dropout(self.relu(self.bn5(self.fc2(x))))
        x = self.fc3(x)
        return x


import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR

# Assuming dataloader is already defined and loaded with data
# Example: dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = PointNet().to(device)
optimizer = optim.Adam(net.parameters(), lr=learning_rate)
criterion = nn.BCEWithLogitsLoss()
scheduler = StepLR(optimizer, step_size=20, gamma=0.5)

num_epochs = 100
patience = 10  # For early stopping
best_loss = float('inf')
early_stop_counter = 0

for epoch in range(num_epochs):
    net.train()
    running_loss = 0.0

    for i, (inputs, labels) in enumerate(dataloader, 0):
        inputs = inputs.transpose(2, 1).to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = net(inputs).squeeze()

        if torch.isnan(outputs).any():
            print("NaN values found in outputs")
            continue

        loss = criterion(outputs, labels)

        if torch.isnan(loss).any():
            print("NaN values found in loss")
            continue

        loss.backward()
        torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=1.0)
        optimizer.step()

        running_loss += loss.item()
        if i % 10 == 9:
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 10:.3f}")
            running_loss = 0.0

    # Step the scheduler
    scheduler.step()

    # Early stopping
    if running_loss < best_loss:
        best_loss = running_loss
        early_stop_counter = 0
    else:
        early_stop_counter += 1

    if early_stop_counter >= patience:
        print("Early stopping triggered")
        break

print("Finished Training")
